In [1]:
# 1.获取数据集
# 2.基本数据处理
# 2.1 缩小数据范围
# 2.2 选择时间特征
# 2.3 去掉签到较少的地方
# 2.4 确定特征值和目标值
# 2.5 分割数据集
# 3.特征工程 -- 特征预处理(标准化)
# 4.机器学习 -- knn+cv
# 5.模型评估

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [3]:
#1.获取数据集
data = pd.read_csv('C:/Users/Administrator/Desktop/ml_data/day05资料/2.code/data/FBlocation/train.csv')

In [4]:
data.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [5]:
data.describe()

,row_id,x,y,accuracy,time,place_id
count,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07
mean,1.455901e+07,4.999770e+00,5.001814e+00,8.284912e+01,4.170104e+05,5.493787e+09
std,8.405649e+06,2.857601e+00,2.887505e+00,1.147518e+02,2.311761e+05,2.611088e+09
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000016e+09
25%,7.279505e+06,2.534700e+00,2.496700e+00,2.700000e+01,2.030570e+05,3.222911e+09
50%,1.455901e+07,5.009100e+00,4.988300e+00,6.200000e+01,4.339220e+05,5.518573e+09
75%,2.183852e+07,7.461400e+00,7.510300e+00,7.500000e+01,6.204910e+05,7.764307e+09
max,2.911802e+07,1.000000e+01,1.000000e+01,1.033000e+03,7.862390e+05,9.999932e+09


In [6]:
data.shape

(29118021, 6)

In [7]:
# 2.基本数据处理
# 2.1 缩小数据范围
partial_data = data.query('x>2.0 & x<2.5 & y>2.0 & y<2.5')

In [8]:
# 2.2选择时间特征
time = pd.to_datetime(partial_data.time,unit='s')
time

163        1970-01-08 18:02:17
310        1970-01-03 17:11:59
658        1970-01-06 19:32:23
1368       1970-01-04 16:50:22
1627       1970-01-07 21:18:04
                   ...        
29116142   1970-01-03 12:31:26
29116267   1970-01-04 15:19:20
29116295   1970-01-01 20:49:14
29116475   1970-01-03 09:17:37
29117203   1970-01-02 20:34:43
Name: time, Length: 71664, dtype: datetime64[ns]

In [9]:
time = pd.DatetimeIndex(time)
time

DatetimeIndex(['1970-01-08 18:02:17', '1970-01-03 17:11:59',
               '1970-01-06 19:32:23', '1970-01-04 16:50:22',
               '1970-01-07 21:18:04', '1970-01-02 03:14:59',
               '1970-01-07 03:45:16', '1970-01-05 03:28:43',
               '1970-01-01 18:59:03', '1970-01-09 07:50:12',
               ...
               '1970-01-09 20:03:34', '1970-01-08 09:26:50',
               '1970-01-07 04:45:59', '1970-01-07 22:36:18',
               '1970-01-06 23:29:43', '1970-01-03 12:31:26',
               '1970-01-04 15:19:20', '1970-01-01 20:49:14',
               '1970-01-03 09:17:37', '1970-01-02 20:34:43'],
              dtype='datetime64[ns]', name='time', length=71664, freq=None)

In [10]:
partial_data['day'] = time.day
partial_data['hour'] = time.hour
partial_data['weekday'] = time.weekday
partial_data

d:\program files (x86)\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
d:\program files (x86)\python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\program files (x86)\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

,row_id,x,y,accuracy,time,place_id,day,hour,weekday
163,163,2.1663,2.3755,84,669737,3869813743,8,18,3
310,310,2.3695,2.2034,3,234719,2636621520,3,17,5
658,658,2.3236,2.1768,66,502343,7877745055,6,19,1
1368,1368,2.2613,2.3392,73,319822,9775192577,4,16,6
1627,1627,2.3331,2.0011,66,595084,6731326909,7,21,2
...,...,...,...,...,...,...,...,...,...
29116142,29116142,2.0804,2.0657,168,217886,1247398579,3,12,5
29116267,29116267,2.4309,2.4646,33,314360,1951613663,4,15,6
29116295,29116295,2.1797,2.1707,89,74954,4724115005,1,20,3
29116475,29116475,2.3924,2.2704,62,206257,2819110495,3,9,5


In [23]:
# 2.3 去掉签到较少的地方
place_count = partial_data.groupby('place_id').count()

In [20]:
place_count = place_count[place_count['row_id']>3]
place_count

,row_id,x,y,accuracy,time,day,hour,weekday
place_id,,,,,,,,
1008823061,4,4,4,4,4,4,4,4
1025585791,21,21,21,21,21,21,21,21
1026507711,220,220,220,220,220,220,220,220
1032417180,10,10,10,10,10,10,10,10
1040557418,123,123,123,123,123,123,123,123
...,...,...,...,...,...,...,...,...
9966115681,15,15,15,15,15,15,15,15
9970566102,5,5,5,5,5,5,5,5
9983648790,1056,1056,1056,1056,1056,1056,1056,1056


In [22]:
partial_data = partial_data[partial_data['place_id'].isin(place_count.index)]
partial_data

,row_id,x,y,accuracy,time,place_id,day,hour,weekday
163,163,2.1663,2.3755,84,669737,3869813743,8,18,3
310,310,2.3695,2.2034,3,234719,2636621520,3,17,5
658,658,2.3236,2.1768,66,502343,7877745055,6,19,1
1368,1368,2.2613,2.3392,73,319822,9775192577,4,16,6
1627,1627,2.3331,2.0011,66,595084,6731326909,7,21,2
...,...,...,...,...,...,...,...,...,...
29116142,29116142,2.0804,2.0657,168,217886,1247398579,3,12,5
29116267,29116267,2.4309,2.4646,33,314360,1951613663,4,15,6
29116295,29116295,2.1797,2.1707,89,74954,4724115005,1,20,3
29116475,29116475,2.3924,2.2704,62,206257,2819110495,3,9,5


In [25]:
# 2.4 确定特征值和目标值
x = partial_data[["x", "y", "accuracy", "day", "hour", "weekday"]]
y = partial_data['place_id']

In [27]:
# 2.5 分割数据集
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=2)

In [29]:
# 3.特征工程--特征预处理(标准化)
# 3.1 实例化一个转换器
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [56]:
# 4.机器学习--knn+cv
# 4.1 实例化一个估计器
estimator = KNeighborsClassifier()
# 4.2 调用gridsearchCV
param_grid = {'n_neighbors': [3, 5, 7,9]}
estimator = GridSearchCV(estimator, param_grid=param_grid, cv=5,n_jobs=-1)
#4.3 模型训练
estimator.fit(x_train, y_train)

d:\program files (x86)\python\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 7, 9]})

In [57]:
# 5.模型评估
# 5.1 基本评估方式
score = estimator.score(x_test, y_test)
print("最后预测的准确率为:\n", score)

最后预测的准确率为:
 0.003970259149642677


In [58]:
y_predict = estimator.predict(x_test)
print("最后的预测值为:\n", y_predict)
print("预测值和真实值的对比情况:\n", y_predict == y_test)

最后的预测值为:
 [1207701286 4907914053 1207701286 ... 4907914053 1207701286 1207701286]
预测值和真实值的对比情况:
 21112721    False
15305275    False
2286447     False
2683519     False
276963      False
            ...  
13273318    False
2638260     False
7873410     False
15127898    False
359708      False
Name: place_id, Length: 13853, dtype: bool


In [59]:
# 5.2 使用交叉验证后的评估方式
print("在交叉验证中验证的最好结果:\n", estimator.best_score_)
print("最好的参数模型:\n", estimator.best_estimator_)
print("每次交叉验证后的验证集准确率结果和训练集准确率结果:\n",estimator.cv_results_)

在交叉验证中验证的最好结果:
 0.35646348987898774
最好的参数模型:
 KNeighborsClassifier()
每次交叉验证后的验证集准确率结果和训练集准确率结果:
 {'mean_fit_time': array([0.14884305, 0.13686376, 0.13856869, 0.13185067]), 'std_fit_time': array([0.0076401 , 0.00872406, 0.01317514, 0.00939497]), 'mean_score_time': array([0.60500898, 0.62817054, 0.69344392, 0.69736061]), 'std_score_time': array([0.0103546 , 0.01360473, 0.02779588, 0.01654668]), 'param_n_neighbors': masked_array(data=[3, 5, 7, 9],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 7}, {'n_neighbors': 9}], 'split0_test_score': array([0.34584499, 0.35802581, 0.35486782, 0.35044663]), 'split1_test_score': array([0.34362029, 0.35345605, 0.35462913, 0.34858329]), 'split2_test_score': array([0.34298863, 0.36058473, 0.35796788, 0.35327558]), 'split3_test_score': array([0.34497383, 0.35652409, 0.35435842, 0.35589244]), 'split4_test_score': array([0.34064248, 0.3537267